In [ ]:

import json
import tweepy
import pandas as pd
from tweepy import OAuthHandler
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import time
import re

#Analysis imports
import glob
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import Counter
import numpy as np
from nltk.stem import WordNetLemmatizer
from datetime import datetime

analyzer = SentimentIntensityAnalyzer()

In [3]:
con_key = 'uz4wSx2vt2PlKbWCOlIpKbBZo'
con_secret = 'lq61FxYRvJPwLjjRlUf0rSTsdXf86iijHYeGfrUdi7EhZzWsWv'
acc_token = '1038510225786122247-zLnTCjChQ0rLUHevueL0CN4ygdY1Me'
acc_token_secret = 'W7lqUevzXRpzI4zQA6niMDR85r9SCo3sCQGaPA4wg6DS6'

In [4]:
auth = OAuthHandler(con_key, con_secret)
auth.set_access_token(acc_token, acc_token_secret)
new = tweepy.API(auth)

In [41]:
bagOwords = ['election' , 'senate' , 'race' , 'november' , '6' , 'gop' , 'dnc' , 'win' , 'lose']
analyzer = SentimentIntensityAnalyzer()
def Senter(tweets , Cadidate , DF):
    for tweet in tweets:
        sentiment = '0'
        retweet = '0'
        relivent = 0
        retweeted_count = 0
        
        if tweet.full_text[:2] == 'RT':
            retweet = '1'
            retweeted_count+=1
        
        lower_txt = tweet.full_text.lower()
        for word in bagOwords:
            if word in lower_txt:
                relivent = 1
        
        if tweet.lang == 'en':
            #send to english sentiment analyser
            sentiment = analyzer.polarity_scores(tweet.full_text)
        else:
            #send to this sentiment analyser
            sentiment = analyzer.polarity_scores(tweet.full_text)
        DF = DF.append({'Candidate': '%s' % (Cadidate) , 'Time' : tweet.created_at ,
               'Text': tweet.full_text , 'UserLocation': tweet.user.location , 
               'Coordinates': tweet.coordinates , 
               'IsReTweet': retweet ,'Language' : tweet.lang , 'Sentiment': sentiment ,
               'Relivent' : relivent } , ignore_index = True)

    return DF

In [42]:
#Keep in mind we are only pulling english tweets
def Searcher(Handle , Cadidate , GeoCode , DF):
    time.sleep(60 * 5)
    tweets = new.search(q='%s' % (Handle), lang = 'en' , count = consume_count  , geocode = '%s' % (GeoCode) , tweet_mode = 'extended')     
    if len(tweets) == 0:
        time.sleep(60 * 5)
        tweets = new.search(q='%s' % (Handle), lang = 'en' , count = consume_count ,  geocode = '%s' % (GeoCode) , tweet_mode = 'extended')    
    DF = Senter(tweets , Cadidate , DF)
    return DF

In [ ]:
columns = ['Candidate' , 'Time' , 'Text' , 'UserLocation' , 'Coordinates' , 'IsReTweet' , 'Language' , 'Sentiment' , 'Relivent' ]
FL_Radius = '27.967042,-81.471955,320km'
AZ_Radius = '34.383026,-111.694738,400km'
consume_count = 5
analyzer = SentimentIntensityAnalyzer()
FL = pd.DataFrame(columns = columns)
AZ = pd.DataFrame(columns = columns)

while True:
    
    if (time.localtime().tm_hour == 25):
        AZ_path_name = '/Users/pbfile/Desktop/Politter/AZOUT' + str(datetime.now())+'.csv'
        FL_path_name = '/Users/pbfile/Desktop/Politter/FLOUT'+ str(datetime.now())+'.csv'
        FL.to_csv(FL_path_name , sep = ',')
        AZ.to_csv(AZ_path_name , sep = ',')

        
    FL = Searcher("@FLGovScott" , 'GOP:RICK SCOTT' , FL_Radius , FL)
    FL = Searcher("@SenBillNelson" , 'DEM:BILL NELSON' , FL_Radius , FL)
    
    AZ = Searcher("@RepMcSally" , 'GOP:MARTHA MCSALLY' , AZ_Radius , AZ)
    AZ = Searcher("@RepSinema" , 'DEM:KYRSTEN SINEMA' , AZ_Radius , AZ)
    

In [50]:
time.localtime().tm_hour

13

# EXPORT WHATEVER IS IN THE DATAFRAMES 

In [59]:
AZ_path_name = '/Users/pbfile/Desktop/Politter/AZOUT' + str(datetime.now())+'.csv'
FL_path_name = '/Users/pbfile/Desktop/Politter/FLOUT'+ str(datetime.now())+'.csv'
FL.to_csv(FL_path_name , sep = ',')
AZ.to_csv(AZ_path_name , sep = ',')

In [35]:
FL.Time.min()

Timestamp('2018-10-24 03:11:37')

In [36]:
FL.Time.max()

Timestamp('2018-10-24 12:25:11')

In [58]:
FL.shape

(279, 9)

In [55]:
AZ.shape

(211, 9)

In [21]:
FL

,Candidate,Time,Text,UserLocation,Coordinates,IsReTweet,Language,Sentiment,Relivent
0,GOP:RICK SCOTT,2018-10-29 18:19:56,RT @FLGovScott: @SalArmyFL has provided more t...,"PUERTO RICO,U.S.A.",None,1,en,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0
1,DEM:BILL NELSON,2018-10-29 18:27:49,.@SenBillNelson every voter deserves to know w...,"Tampa, FL",None,0,en,"{'neg': 0.0, 'neu': 0.914, 'pos': 0.086, 'comp...",1
2,DEM:BILL NELSON,2018-10-29 18:26:16,RT @Doodisgirl: Shocked to see Democrats like ...,"Long Island, New York",None,1,en,"{'neg': 0.232, 'neu': 0.664, 'pos': 0.104, 'co...",0
3,DEM:BILL NELSON,2018-10-29 18:26:02,RT @EdKrassen: Here are Democratic Senators wh...,"Thunder Bay, Ontario, Canada",None,1,en,"{'neg': 0.0, 'neu': 0.782, 'pos': 0.218, 'comp...",0
4,DEM:BILL NELSON,2018-10-29 18:24:03,RT @EdKrassen: Please support these Democrats ...,"Pittsburgh, PA",None,1,en,"{'neg': 0.0, 'neu': 0.726, 'pos': 0.274, 'comp...",1
5,DEM:BILL NELSON,2018-10-29 18:23:39,RT @Doodisgirl: Shocked to see Democrats like ...,"Proud American, Auburn Tiger. War Eagle!",None,1,en,"{'neg': 0.232, 'neu': 0.664, 'pos': 0.104, 'co...",0


In [ ]:
test = 